In [2]:
import json, os, numpy as np, tqdm, requests
from collections import defaultdict
from rdkit import Chem

In [2]:
with open("ALL_GNPS.json") as f:
    all_gnps = json.load(f)
del_list = []
for i in tqdm.tqdm(range(len(all_gnps))):
    if all_gnps[i]["peaks_json"] == "[]":
        del_list.append(i)
print("Data Len: ", len(all_gnps))
for i in reversed(del_list):
    del all_gnps[i]
print("Data Len after: ", len(all_gnps))


100%|██████████| 210569/210569 [00:00<00:00, 1514851.78it/s]

Data Len:  210569
Data Len after:  209654


In [3]:
# extracting the only needed keys, used for unsupervised learning (e.x. invalid SMILES)
allowed = set(["spectrum_id", "peaks_json", "Charge", "Ion_Mode", "Adduct", "Smiles", "SpectrumID", "Formula_smiles", "url", "Precursor_MZ", "Compound_Name"])
for x in tqdm.tqdm(all_gnps):
    to_delete = []
    for key in x:
        if key not in allowed: 
            to_delete.append(key)
    for key in to_delete:
        del x[key]
with open("ALL_GNPS_trimmedkeys_removedempty.json", "w") as f:
    json.dump(all_gnps, f)

100%|██████████| 209654/209654 [00:01<00:00, 142918.91it/s]


In [5]:
with open("ALL_GNPS_trimmedkeys_removedempty.json", "r") as f:
    all_gnps = json.load(f)

In [6]:
# Remove empty smiles or non-parseable smiles
exclude = set(["N/A", " ", "", "NO DATA", "NA", 'no data', "n/a", "na"])
to_delete = []
for idx, sample in tqdm.tqdm(enumerate(all_gnps)):
    smiles = sample["Smiles"]
    if smiles in exclude:
        to_delete.append(idx)
        continue
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        to_delete.append(idx)
print(f"Deleting {len(to_delete)} samples")
for item in reversed(to_delete):
    del all_gnps[item]
with open("ALL_GNPS_trimmedkeys_filteredsmiles.json", "w") as f:
    json.dump(all_gnps, f)

209654it [00:18, 11529.92it/s] 


Deleting 111036 samples


In [3]:
with open("ALL_GNPS_trimmedkeys_filteredsmiles.json", "r") as f:
    all_gnps = json.load(f)

In [4]:
# scrape classifications
uniques = set()
for sample in tqdm.tqdm(all_gnps):
    sm = sample["Smiles"]
    if sm in uniques:
        continue
    mol = Chem.MolFromSmiles(sm)
    if mol is not None:
        uniques.add(sm)
print(f"Uniques: {len(uniques)}")
with open("ALL_SMILES.json", "w") as f:
    json.dump(list(uniques), f)

100%|██████████| 98618/98618 [00:04<00:00, 19830.69it/s]

Uniques: 22695


In [6]:
with open("ALL_SMILES.json", "r") as f:
    uniques = set(json.load(f))